In [2]:
# Manipulation des données
import pandas as pd
import numpy as np
import ast

# Gestion des fichiers et des chemins
import glob

# Prétraitement des images (TensorFlow / Keras)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input

# Prétraitement des étiquettes
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [4]:
df=pd.read_csv(r"C:\Users\mauge\Openclassrooms\Projet 7 - Git repository\Veille\Data\flipkart_com-ecommerce_sample_1050.csv")

# Récupération des catégories
def extract_first_category(category_string):
    # Convertir la chaîne de caractères en liste
    category_list = ast.literal_eval(category_string)
    
    # Sélectionner la première valeur de la liste
    full_category = category_list[0]
    
    # Sélectionner la première partie avant '>>'
    first_category = full_category.split('>>')[0].strip()
    
    return first_category

df['main_category'] = df['product_category_tree'].apply(extract_first_category)
list_labels = df['main_category'].unique().tolist()
df.head(1)
df.shape
list_labels


path = r"C:\Users\mauge\Openclassrooms\Projet 7 - Git repository\Veille\Data\Images"

data_path = glob.glob(path + '/*.jp*')

def data_fct(path) :
    list_photos = [file for file in path]
    print(len(list_photos))
    data = pd.DataFrame()
    data["image_path"] = list_photos
    return data

data = data_fct(data_path)

data.head(5)
data['file_id'] = data['image_path'].apply(lambda x: x.split('\\')[-1].split('.')[0])
df.rename(columns={'uniq_id': 'file_id'}, inplace=True)
data = pd.merge(data, df, on='file_id', how='left')

1050


In [5]:
data.head(1)

,image_path,file_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,main_category
0,C:\Users\mauge\Openclassrooms\Projet 7 - Git r...,009099b1f6e1e8f893ec29a7023153c4,2016-04-24 18:34:50 +0000,http://www.flipkart.com/palito-plo-166-analog-...,"palito PLO 166 Analog Watch - For Girls, Women","[""Watches >> Wrist Watches >> palito Wrist Wat...",WATEHZPFAPQKUASQ,1500.0,199.0,009099b1f6e1e8f893ec29a7023153c4.jpg,False,Specifications of palito PLO 166 Analog Watch ...,No rating available,No rating available,palito,"{""product_specification""=>[{""key""=>""Type"", ""va...",Watches


# SWIN Transformer

In [ ]:
import torch
from transformers import AutoImageProcessor, SwinForImageClassification
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score


In [38]:
from sklearn.preprocessing import LabelEncoder

# Initialiser le LabelEncoder
label_encoder = LabelEncoder()

# Ajouter une colonne encodée
data['label_encoded'] = label_encoder.fit_transform(data['main_category'])

data['label_encoded'].value_counts()


label_encoded
6    150
5    150
4    150
1    150
2    150
3    150
0    150
Name: count, dtype: int64

In [44]:
data.head(1)

,image_path,file_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,main_category,label_encoded
0,C:\Users\mauge\Openclassrooms\Projet 7 - Git r...,009099b1f6e1e8f893ec29a7023153c4,2016-04-24 18:34:50 +0000,http://www.flipkart.com/palito-plo-166-analog-...,"palito PLO 166 Analog Watch - For Girls, Women","[""Watches >> Wrist Watches >> palito Wrist Wat...",WATEHZPFAPQKUASQ,1500.0,199.0,009099b1f6e1e8f893ec29a7023153c4.jpg,False,Specifications of palito PLO 166 Analog Watch ...,No rating available,No rating available,palito,"{""product_specification""=>[{""key""=>""Type"", ""va...",Watches,6


In [39]:
from sklearn.model_selection import train_test_split

# Split en train et validation
train_data, val_data = train_test_split(data, test_size=0.2, stratify=data['main_category'], random_state=42)

print(f"Nombre d'images pour l'entraînement : {len(train_data)}")
print(f"Nombre d'images pour la validation : {len(val_data)}")


Nombre d'images pour l'entraînement : 840
Nombre d'images pour la validation : 210


In [40]:
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, dataframe, processor):
        self.dataframe = dataframe
        self.processor = processor

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_path = self.dataframe.iloc[idx]['image_path']
        label = self.dataframe.iloc[idx]['label_encoded']

        # Charger et redimensionner l'image
        img = Image.open(image_path).convert('RGB')
        img = img.resize((224, 224))  # Taille standard pour Swin Transformer

        # Prétraitement
        processed = self.processor(images=img, return_tensors="pt")

        return processed['pixel_values'].squeeze(0), label


In [43]:
# Charger le processeur pour Swin Transformer
from transformers import AutoImageProcessor

processor = AutoImageProcessor.from_pretrained("microsoft/swin-tiny-patch4-window7-224")

# Création des datasets
train_dataset = CustomDataset(train_data, processor)
val_dataset = CustomDataset(val_data, processor)


In [24]:
# Vérifier les labels encodés
print("Labels originaux :", data['main_category'].unique())
print("Labels encodés :", label_encoder.transform(data['main_category'].unique()))


Labels originaux : ['Watches' 'Kitchen & Dining' 'Home Furnishing' 'Beauty and Personal Care'
 'Computers' 'Home Decor & Festive Needs' 'Baby Care']
Labels encodés : [6 5 4 1 2 3 0]


In [51]:
for images, labels in train_loader:
    print(f"Image batch shape: {images.shape}")
    print(f"Label batch shape: {labels.shape}")
    break


Image batch shape: torch.Size([32, 3, 224, 224])
Label batch shape: torch.Size([32])


In [52]:
from torch.utils.data import DataLoader

# Création des DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Vérification des classes
num_classes = len(train_data['main_category'].unique())
print("Nombre de classes :", num_classes)



Nombre de classes : 7


In [53]:
from transformers import SwinForImageClassification
import torch.nn as nn

# Charger le modèle pré-entraîné avec gestion des tailles incompatibles
model_name = "microsoft/swin-tiny-patch4-window7-224"
model = SwinForImageClassification.from_pretrained(
    model_name,
    ignore_mismatched_sizes=True
)

# Remplacer la couche de classification
num_classes = 7  # Nombre de classes de votre tâche
model.classifier = nn.Linear(in_features=model.classifier.in_features, out_features=num_classes)

# Déplacer le modèle sur le GPU ou CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



SwinForImageClassification(
  (swin): SwinModel(
    (embeddings): SwinEmbeddings(
      (patch_embeddings): SwinPatchEmbeddings(
        (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): SwinEncoder(
      (layers): ModuleList(
        (0): SwinStage(
          (blocks): ModuleList(
            (0): SwinLayer(
              (layernorm_before): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (attention): SwinAttention(
                (self): SwinSelfAttention(
                  (query): Linear(in_features=96, out_features=96, bias=True)
                  (key): Linear(in_features=96, out_features=96, bias=True)
                  (value): Linear(in_features=96, out_features=96, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
                )
                (output): SwinSelfOutput(
        

In [54]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

# Définition de l'optimiseur et de la fonction de perte
optimizer = Adam(model.parameters(), lr=2e-5)
loss_fn = CrossEntropyLoss()


In [55]:
def validate_model(model, val_loader, loss_fn, device):
    model.eval()  # Mettre le modèle en mode évaluation
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():  # Pas de calcul des gradients en validation
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images).logits
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()
            
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())
    
    val_loss = total_loss / len(val_loader)
    val_accuracy = accuracy_score(all_labels, all_preds)
    return val_loss, val_accuracy


In [56]:
import torch
from sklearn.metrics import accuracy_score  # Pour calculer l'accuracy

def train_model(model, train_loader, val_loader, optimizer, loss_fn, device, epochs=5):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        all_preds = []
        all_labels = []

        print(f"Epoch {epoch+1}/{epochs}")
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images).logits
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

            # Afficher les logs tous les 10 batches
            if (batch_idx + 1) % 10 == 0:
                print(f"Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")
        
        train_loss = total_loss / len(train_loader)
        train_accuracy = accuracy_score(all_labels, all_preds)

        # Validation
        val_loss, val_accuracy = validate_model(model, val_loader, loss_fn, device)

        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")




In [57]:
train_model(model, train_loader, val_loader, optimizer, loss_fn, device, epochs=5)


Epoch 1/5
Batch 10/27, Loss: 1.7559


c:\Users\mauge\Openclassrooms\Projet 7 - Git repository\gradio_env\Lib\site-packages\PIL\Image.py:3406: DecompressionBombWarning: Image size (93680328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Batch 20/27, Loss: 1.3872
Train Loss: 1.6206, Train Accuracy: 0.4619, Val Loss: 1.1872, Val Accuracy: 0.7095
Epoch 2/5
Batch 10/27, Loss: 0.8169


c:\Users\mauge\Openclassrooms\Projet 7 - Git repository\gradio_env\Lib\site-packages\PIL\Image.py:3406: DecompressionBombWarning: Image size (93680328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Batch 20/27, Loss: 0.6426
Train Loss: 0.7047, Train Accuracy: 0.8548, Val Loss: 0.6768, Val Accuracy: 0.7762
Epoch 3/5
Batch 10/27, Loss: 0.4175


...

Batch 20/27, Loss: 0.1243
Train Loss: 0.3277, Train Accuracy: 0.9155, Val Loss: 0.6126, Val Accuracy: 0.8048
Epoch 4/5
Batch 10/27, Loss: 0.2048


c:\Users\mauge\Openclassrooms\Projet 7 - Git repository\gradio_env\Lib\site-packages\PIL\Image.py:3406: DecompressionBombWarning: Image size (93680328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Batch 20/27, Loss: 0.1261
Train Loss: 0.1734, Train Accuracy: 0.9655, Val Loss: 0.6184, Val Accuracy: 0.8190
Epoch 5/5


c:\Users\mauge\Openclassrooms\Projet 7 - Git repository\gradio_env\Lib\site-packages\PIL\Image.py:3406: DecompressionBombWarning: Image size (93680328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Batch 10/27, Loss: 0.0904
Batch 20/27, Loss: 0.0514
Train Loss: 0.0943, Train Accuracy: 0.9869, Val Loss: 0.6189, Val Accuracy: 0.8095


In [ ]:
def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Prédictions
            outputs = model(images).logits
            preds = torch.argmax(outputs, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

evaluate_model(model, test_loader, device)


In [ ]:
# Sauvegarder le modèle
model.save_pretrained("swin_model")

# Recharger le modèle
model = SwinForImageClassification.from_pretrained("swin_model")
model.to(device)


# Vision Transformer (ViT)

In [ ]:
import torch
from transformers import AutoImageProcessor, ViTForImageClassification
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import numpy as np
from PIL import Image
import pandas as pd


In [36]:
from sklearn.preprocessing import LabelEncoder

# Initialiser le LabelEncoder
label_encoder = LabelEncoder()

# Ajouter une colonne encodée
data['label_encoded'] = label_encoder.fit_transform(data['main_category'])

data['label_encoded'].value_counts()


label_encoded
6    150
5    150
4    150
1    150
2    150
3    150
0    150
Name: count, dtype: int64

In [37]:
from sklearn.model_selection import train_test_split

# Split en train et validation
train_data, val_data = train_test_split(data, test_size=0.2, stratify=data['main_category'], random_state=42)

print(f"Nombre d'images pour l'entraînement : {len(train_data)}")
print(f"Nombre d'images pour la validation : {len(val_data)}")

Nombre d'images pour l'entraînement : 840
Nombre d'images pour la validation : 210


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, processor):
        self.dataframe = dataframe
        self.processor = processor

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Charger l'image
        image_path = self.dataframe.iloc[idx]['image_path']
        label = self.dataframe.iloc[idx]['label_encoded']
        img = Image.open(image_path).convert('RGB')
        
        # Prétraiter l'image
        processed = self.processor(images=img, return_tensors="pt")
        return processed['pixel_values'].squeeze(0), label


In [ ]:
from transformers import AutoImageProcessor, ViTForImageClassification

# Charger le processeur et le modèle
model_name = "google/vit-base-patch16-224"
processor = AutoImageProcessor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels=num_classes
)

# Déplacer le modèle sur le GPU ou CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
# Créer les datasets
train_dataset = CustomDataset(train_data, processor)
val_dataset = CustomDataset(val_data, processor)

# Créer les DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
def train_model(model, train_loader, optimizer, loss_fn, device, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        all_preds = []
        all_labels = []

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images).logits
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

        train_accuracy = accuracy_score(all_labels, all_preds)
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.4f}")
